<a href="https://colab.research.google.com/github/mafaldasalomao/multilabel_classification_pytorch/blob/main/multilabel_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import suppporting libraries
import tarfile
import urllib.request as urllib2
import os
import cv2
from os import listdir
from os.path import isfile, join
import re
#Import deep learning libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms, models
import torchvision.models as models
#Import data analytics libraries
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import scipy.io
import pandas as pd
import seaborn as sns
#Import image visualization libraries
from PIL import *
from PIL import ImageFile
from PIL import Image
#System settings
ImageFile.LOAD_TRUNCATED_IMAGES = True
os.environ['WANDB_CONSOLE'] = 'off'
#Coloring for print outputs
class color:
   RED = '\033[91m'
   BOLD = '\033[1m'
   END = '\033[0m'

In [11]:
!unzip convex.zip -d /content/temp

Archive:  convex.zip
   creating: /content/temp/convex/
  inflating: /content/temp/convex/Cov-Atlas+(44).gif  
  inflating: /content/temp/convex/Cov-Atlas+(45).gif  
  inflating: /content/temp/convex/Cov-Atlas-+(43).gif  
  inflating: /content/temp/convex/Cov-Atlas-Day+1.gif  
  inflating: /content/temp/convex/Cov-Atlas-Day+2.gif  
  inflating: /content/temp/convex/Cov-Atlas-Day+3.gif  
  inflating: /content/temp/convex/Cov-Atlas-Day+4.gif  
  inflating: /content/temp/convex/Cov-Atlas-suspectedCovid.mp4  
  inflating: /content/temp/convex/Cov-clarius.gif  
  inflating: /content/temp/convex/Cov-clarius3.gif  
  inflating: /content/temp/convex/Cov-grep-7453.mp4  
  inflating: /content/temp/convex/Cov-grep-7505.mp4  
  inflating: /content/temp/convex/Cov-grep-7507.mp4  
  inflating: /content/temp/convex/Cov-grep-7510.mp4  
  inflating: /content/temp/convex/Cov-grep-7511.mp4  
  inflating: /content/temp/convex/Cov-grep-7525.mp4  
  inflating: /content/temp/convex/Cov-grep-7543.mp4  
  infl

In [14]:
path='/content/temp/convex'
annotation_path = "dataset_metadata.csv"
videos_list = os.listdir(path)

In [15]:
columns = ['Filename', 'Effusion', 'Consolidations', 'B-lines', 'A-lines', 'Pleural_line_irregularities', 'Air_bronchogram', 'Label']
base = pd.read_csv(annotation_path, usecols=columns)

In [16]:
base = base[base['Effusion'] != 'n/A']
base = base[base['Consolidations'] != 'n/A']
base = base[base['B-lines'] != 'n/A']
base = base[base['A-lines']  != 'n/A']
base = base[base['Pleural_line_irregularities']  != 'n/A']
base = base[base['Air_bronchogram']  != 'n/A']
base = base[base['Label']  != 'n/A']
#base = base[base['Label']  != 'regular']
#base = base[base['Label']  != 'Regular']
base['Label']= base['Label'].replace(['regular'], 'Regular')

base['Effusion'] = pd.to_numeric(base['Effusion'])
base.head()

,Filename,Label,Effusion,Consolidations,B-lines,A-lines,Pleural_line_irregularities,Air_bronchogram
0,Cov-Butterfly-COVID Lung 2,COVID-19,1.0,0.0,1.0,0.0,1.0,0.0
1,Cov-Butterfly-Skip Lesion,COVID-19,0.0,1.0,1.0,1.0,1.0,0.0
2,Cov-Butterfly-Confluent B lines,COVID-19,0.0,0.0,1.0,0.0,0.0,0.0
3,Reg-Butterfly-Normal Lung A lines,Regular,0.0,0.0,0.0,1.0,0.0,0.0
4,Reg-Butterfly-Normal Lung_Example 2,Regular,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
base  = base.dropna(axis = 0)
base.shape

(346, 8)

In [18]:
metadata = base;
metadata['Filename']

0               Cov-Butterfly-COVID Lung 2
1                Cov-Butterfly-Skip Lesion
2          Cov-Butterfly-Confluent B lines
3        Reg-Butterfly-Normal Lung A lines
4      Reg-Butterfly-Normal Lung_Example 2
                      ...                 
344                Cov-B_ConvexProb_score1
345               Cov-C_Convex_Prob_score2
346               Cov-D_Convex_Prob_score3
347                Cov-C_ConvexProb_score3
348                    Cov-MSU-SkipLesions
Name: Filename, Length: 346, dtype: object

In [19]:
#"Cov-Butterfly-COVID Lung 2" in metadata['Filename']
teste = metadata.loc[metadata.Filename == "Reg-Atlas-lungcurtain"]
len(teste)

1

In [20]:
effusion_list = []
consolid_list = []
aline_list = []
bline_list = []
pli_list = []
airb_list = []
filenames = []
y_labels = []
num_imgs_per_video = 10

In [21]:
def create_mydataset():
    for filename in videos_list:
        count=0
        #print(filename)
        vid_cap = cv2.VideoCapture(path+'/'+filename)
        total_frame_count= int(vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        step = total_frame_count/num_imgs_per_video
        success, image = vid_cap.read()
        metadata_cap = metadata.loc[metadata.Filename == filename.split('.')[0]]
        if len(metadata_cap) >0:
            while success:
                #print("Reading new frame...", success)
                if ((count+1) % int(step) == 0):
                    #print("Writing new frame...")
                    out = 'dataset/{}_{}.png'.format(filename.split('.')[0], count)
                    #img = tf.keras.utils.load_img(out, target_size = (224, 224, 3))
                    #img = tf.keras.utils.img_to_array(img)
                    #img= img/255.
                    #img = img.transpose(2,0,1)
                    cv2.imwrite(out, image)
                    filenames.append(out)
                    effusion_list.append(metadata['Effusion'][metadata_cap.index[0]])
                    consolid_list.append(metadata['Consolidations'][metadata_cap.index[0]])
                    aline_list.append(metadata['A-lines'][metadata_cap.index[0]])
                    bline_list.append(metadata['B-lines'][metadata_cap.index[0]])
                    pli_list.append(metadata['Pleural_line_irregularities'][metadata_cap.index[0]])
                    airb_list.append(metadata['Air_bronchogram'][metadata_cap.index[0]])
                success, image = vid_cap.read()
                count += 1

In [22]:
create_mydataset()

In [23]:
translation_dict = dict(zip(filenames,list(zip(effusion_list, consolid_list, aline_list, bline_list, pli_list, airb_list))))
translation_dict

{'dataset/Reg_pat2Image_134441_20.png': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 'dataset/Reg_pat2Image_134441_41.png': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 'dataset/Reg_pat2Image_134441_62.png': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 'dataset/Reg_pat2Image_134441_83.png': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 'dataset/Reg_pat2Image_134441_104.png': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 'dataset/Reg_pat2Image_134441_125.png': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 'dataset/Reg_pat2Image_134441_146.png': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 'dataset/Reg_pat2Image_134441_167.png': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 'dataset/Reg_pat2Image_134441_188.png': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 'dataset/Reg_pat2Image_134441_209.png': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 'dataset/Pneu_Pneumonia-LITFL-Ultrasound_crop_1.png': (0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0),
 'dataset/Pneu_Pneumonia-LITFL-Ultrasound_crop_3.png': (0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0),
 'dataset/Pneu_Pneumonia-LITFL-Ultrasound_crop_5.png': (0.0,
  1.0,
  0.0,
  0.0

In [25]:
folder = [x for x in listdir("/content/sample_data")]
int(pocusdata.__len__()*0.8)+ int(pocusdata.__len__()*0.2)

NameError: ignored

In [55]:
class PocusDataset(Dataset):
  
  def __init__(self,pocus_path,filenames,transform,translation_dict):
    self.path = pocus_path
    self.folder = filenames
    self.transform = transform
    self.translation_dict = translation_dict

  def __len__(self):
    return len(self.folder)

  def __getitem__(self,idx):
    img_loc = os.path.join(self.path, self.folder[idx])
    image = Image.open(img_loc).convert('RGB')
    single_img = self.transform(image)

    label1 = int(translation_dict[self.folder[idx]][0])
    label2 = int(translation_dict[self.folder[idx]][1])
    label3 = int(translation_dict[self.folder[idx]][2])
    label4 = int(translation_dict[self.folder[idx]][3])
    label5 = int(translation_dict[self.folder[idx]][4])
    label6 = int(translation_dict[self.folder[idx]][5])

    sample = {'image':single_img, 'labels': {'label_eff':label1, 'label_con':label2, 'label_ali':label3, 'label_bli':label4, 'label_pli':label5, 'label_air':label6}}
    return sample   

In [56]:
#Pre-processing transformations
data_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])

#Getting the data
pocusdata = PocusDataset("/content/",filenames=filenames, transform=data_transforms,translation_dict=translation_dict)
#Split the data in training and testing
train_len = int(pocusdata.__len__()*0.8)
test_len = int(pocusdata.__len__()*0.2)+1
train_set, val_set = torch.utils.data.random_split(pocusdata, [train_len, test_len])

#Create the dataloader for each dataset
train_loader = DataLoader(train_set, batch_size=16, shuffle=True, 
                                num_workers=4, drop_last=True)
test_loader = DataLoader(val_set, batch_size=16, shuffle=False, 
                               num_workers=4, drop_last=True)

In [34]:
sample = next(iter(train_loader))

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f93f59e3170>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
Exception ignored in:     self._shutdown_workers()<function _MultiProcessingDataLoaderIter.__del__ at 0x7f93f59e3170>  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__


<function _MultiProcessingDataLoaderIter.__del__ at 0x7f93f59e3170>  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
Traceback (most recent call last):

      File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
Traceback (most recent call last):
if w.is_alive():
          File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
self._shutdown_workers()    
self._shutdown_workers()
assert self._pare

In [35]:
sample['labels'].keys()

dict_keys(['label_effusion', 'label_consolid', 'label_aline', 'label_bline', 'label_pli', 'label_airb'])

In [36]:
resnet = models.resnet34(pretrained=True)
list(resnet.children())[-3:]

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

[Sequential(
   (0): BasicBlock(
     (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (downsample): Sequential(
       (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
       (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
   )
   (1): BasicBlock(
     (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): Batc

In [37]:
model_wo_fc = nn.Sequential(*(list(resnet.children())[:-1]))

In [38]:
output_sample = model_wo_fc(sample['image'])
print(output_sample.shape)
print(torch.flatten(output_sample, 1).shape)

torch.Size([16, 512, 1, 1])
torch.Size([16, 512])


In [39]:
output_sample_flatten = torch.flatten(output_sample, 1)
brand = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=6)
        )
brand(output_sample_flatten).shape

torch.Size([16, 6])

In [40]:
class MultilabelClassifier(nn.Module):
    def __init__(self, n_labels):
        super().__init__()
        self.resnet = models.resnet34(pretrained=True)
        self.model_wo_fc = nn.Sequential(*(list(self.resnet.children())[:-1]))

        self.eff = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=n_labels)
        )
        self.con = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=n_labels)
        )
        self.ali = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=n_labels)
        )
        self.bli = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=n_labels)
        )
        self.pli = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=n_labels)
        )
        self.air = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=512, out_features=n_labels)
        )


    def forward(self, x):
        x = self.model_wo_fc(x)
        x = torch.flatten(x, 1)

        return {
            'eff': self.eff(x),
            'con': self.con(x),
            'ali': self.ali(x),
            'bli': self.bli(x),
            'pli': self.pli(x),
            'air': self.air(x)
        }

In [53]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultilabelClassifier(6)

In [57]:
def criterion(loss_func,outputs,pictures):
    losses = 0
    for i, key in enumerate(outputs):
        losses += loss_func(outputs[key], pictures['labels'][f'label_{key}'])
        return losses

def training(model,device,lr_rate,epochs,train_loader):
    num_epochs = epochs
    losses = []
  
    checkpoint_losses = []

    optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
    n_total_steps = len(train_loader)

    loss_func = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        for i, pictures in enumerate(train_loader):
            images = pictures['image']
            pictures = pictures

            outputs = model(images)

            loss = criterion(loss_func,outputs, pictures)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % (int(n_total_steps/1)) == 0:
                checkpoint_loss = torch.tensor(losses).mean().item()
                checkpoint_losses.append(checkpoint_loss)
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {checkpoint_loss:.4f}')
    return checkpoint_losses

checkpoint_losses = training(model,device,0.0001,10,train_loader)

Epoch [1/10], Step [76/76], Loss: 0.3024
Epoch [2/10], Step [76/76], Loss: 0.1695
Epoch [3/10], Step [76/76], Loss: 0.1180
Epoch [4/10], Step [76/76], Loss: 0.0941
Epoch [5/10], Step [76/76], Loss: 0.0796
Epoch [6/10], Step [76/76], Loss: 0.0688
Epoch [7/10], Step [76/76], Loss: 0.0598
Epoch [8/10], Step [76/76], Loss: 0.0571
Epoch [9/10], Step [76/76], Loss: 0.0524
Epoch [10/10], Step [76/76], Loss: 0.0476


In [59]:
def validation(model, dataloader, *args):

    all_predictions = torch.tensor([])
    all_true_labels = torch.tensor([])

    with torch.no_grad():
        n_correct = []
        n_class_correct = []
        n_class_samples = []
        n_samples = 0

        for arg in args:
            n_correct.append(len(arg))
            n_class_correct.append([0 for i in range(len(arg))])
            n_class_samples.append([0 for i in range(len(arg))])

        for pictures in dataloader:
            images = pictures['image']
            outputs = model(images)
            labels = [pictures['labels'][picture] for picture in pictures['labels']]

            for i,out in enumerate(outputs):
                _, predicted = torch.max(outputs[out],1)
                n_correct[i] += (predicted == labels[i]).sum().item()

                if i == 0:
                    n_samples += labels[i].size(0)

                for k in range(16):
                    label = labels[i][k]
                    pred = predicted[k]
                    if (label == pred):
                        n_class_correct[i][label] += 1
                    n_class_samples[i][label] += 1
          
    return n_correct,n_samples,n_class_correct,n_class_samples

def class_acc(n_correct,n_samples,n_class_correct,n_class_samples,class_list):
    for i in range(len(class_list)):
        print("-------------------------------------------------")
        acc = 100.0 * n_correct[i] / n_samples
        print(color.BOLD + color.RED + f'Overall class performance: {round(acc,1)} %' + color.END)
        for k in range(len(class_list[i])):
            acc = 100.0 * n_class_correct[i][k] / n_class_samples[i][k]
            print(f'Accuracy of {class_list[i][k]}: {round(acc,1)} %')
    print("-------------------------------------------------")

classes_brand = list(effusion_dict.values())
classes_vehicle_type = list(effusion_dict.values())
classes_epoch = ['2009 and earlier','2010 and later']
class_list = [classes_brand,classes_brand,classes_brand]

n_correct,n_samples,n_class_correct,n_class_samples = validation(model,test_loader,classes_brand,classes_vehicle_type,classes_epoch)

class_acc(n_correct,n_samples,n_class_correct,n_class_samples,class_list)

NameError: ignored